# Configurações iniciais

## inicialização

In [ ]:
import os
import sys

# (1) Caminho raiz do seu projeto (ajuste se precisar)
PROJECT_DIR = "/content/drive/MyDrive/tcc-modelo/3-tcc-demand-forecasting"

# (2) Monta o Drive, se ainda não estiver montado
if not os.path.ismount("/content/drive"):
    from google.colab import drive
    drive.mount("/content/drive")

# (3) Garante que o PROJECT_DIR está no sys.path
if PROJECT_DIR not in sys.path:
    sys.path.append(PROJECT_DIR)

print("Repositório ativo em:", PROJECT_DIR)

## Importações

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from src.data.loader import load_dataset
import os

## leitura das bases

In [ ]:

cfg_path = os.path.join(PROJECT_DIR, "configs/data.yaml")

dfs = load_dataset(cfg_path, dataset="olist", stage="raw")

for name, df in dfs.items():
    print(f"{name}: {df.info()}")
    print("\n" + "="*50 + "\n\n")

## visualização dos datasets que serão utilizados

In [ ]:
datasets_utilizados = ['olist_order_reviews_dataset.csv','olist_order_items_dataset.csv','olist_orders_dataset.csv','olist_products_dataset.csv', 'olist_customers_dataset.csv']

In [ ]:
for name, df in dfs.items():
  if name in datasets_utilizados:
    print(f"{name}: {df.info()}")
    print("\n" + "="*50 + "\n\n")

In [ ]:
for df in datasets_utilizados:
  print(df)
  display(dfs[df].head())

In [ ]:
# diferença entre a data do pedido e a de aprovação
# diferençã entre a data de previsão e a de entrega
# avaliar prazos e preços de frete por região
# um pedido pode ter mais de um item?


## converte datas para o formato correto

In [ ]:
#orders
orders = dfs['olist_orders_dataset.csv']
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])
orders['order_approved_at'] = pd.to_datetime(orders['order_approved_at'])
orders['order_delivered_carrier_date'] = pd.to_datetime(orders['order_delivered_carrier_date'])
orders['order_delivered_customer_date'] = pd.to_datetime(orders['order_delivered_customer_date'])
orders['order_estimated_delivery_date'] = pd.to_datetime(orders['order_estimated_delivery_date'])

# Qualidade dos dados

In [ ]:
qualidade = []
for name, df in dfs.items():
    nul = df.isna().sum().sort_values(ascending=False)
    dup = int(df.duplicated().sum())
    qualidade.append({'dataset': name, 'duplicados': dup})
    print(f"\n--- {name} --- duplicados: {dup}")
    display(nul[nul>0].to_frame('nulos'))

qualidade_df = pd.DataFrame(qualidade)
display(qualidade_df)

# Entendimento da base

## existe diferença significativa entre a data do pedido e a de aprovação do pedido?
- Hipótese: pedidos que demoraram mais a ser processados têm impacto futuro em suas vendas?

In [ ]:
# diferença entre a data do pedido e a de aprovação
orders_diff_purchase_aprov = orders['order_approved_at'] - orders['order_purchase_timestamp']
orders_diff_purchase_aprov.describe()

In [ ]:
#convertendo para horas e visualizando histograma
orders_diff_purchase_aprov_hours = orders_diff_purchase_aprov.dt.total_seconds() / 3600
orders_diff_purchase_aprov_hours.hist(bins=5000, figsize=(8,4))
plt.xlim(0, 48)


75% das vendas são aprovadas em até 14 horas e 34 minutos, que é um tempo considerável. Irei gerar uma feature baseada nisso para avaliar o impacto nas vendas

## avaliando prazos de entrega
- Objetivo de entender qual a distribuição dos prazos de entrega

In [ ]:
# prazo em dias
orders['prazo'] = orders['order_estimated_delivery_date'] - orders['order_purchase_timestamp']
orders['prazo'] = orders['prazo'].dt.days

# pegando a coluna de estado do cliente
customer = dfs['olist_customers_dataset.csv']
df_merge = orders.merge(customer[['customer_id', 'customer_state']], on='customer_id', how='left')

# cria uma funcao para gerar a coluna de região baseada na coluna da abreviação dos estados
def get_region(estado):

    estado = estado.upper()

    regioes = {
        "Norte": ["AC", "AP", "AM", "PA", "RO", "RR", "TO"],
        "Nordeste": ["AL", "BA", "CE", "MA", "PB", "PE", "PI", "RN", "SE"],
        "Centro-Oeste": ["DF", "GO", "MT", "MS"],
        "Sudeste": ["ES", "MG", "RJ", "SP"],
        "Sul": ["PR", "RS", "SC"]
    }

    for regiao, estados in regioes.items():
        if estado in estados:
            return regiao

    return "estado inválido"

# gera coluna com região
df_merge['regiao'] = df_merge['customer_state'].apply(get_region)

In [ ]:
orders['prazo'].describe()

In [ ]:
orders['prazo'].hist(bins=200)

Percebe-se que os prazos de entrega dados são muito extensos.
Uma hipótese levantada era se o prazo estimado das entregas poderia ou não ter correlação com as vendas.

In [ ]:
# avaliando o bloxplot dos prazos por região
sns.boxplot(x='regiao', y='prazo', data=df_merge)


no caso da criação de uma feature que avalie o tempo de entrega, cabe olhar separadamente por região. Pois o que é rápido em uma, pode não ser na outra

## avaliação diferença entre a data de previsão e a de entrega
- Hipótese: produtos com entregas mais rápidas têm um maior volume de vendas futuras?

In [ ]:
# diferença entre a data de previsão e a de entrega
orders_diff_estimate_delivered = orders['order_estimated_delivery_date'] - orders['order_delivered_carrier_date']
orders_diff_estimate_delivered.describe()

In [ ]:
#convertendo para horas e visualizando histograma
orders_diff_estimate_delivered = orders_diff_estimate_delivered.dt.total_seconds() / 3600
orders_diff_estimate_delivered.hist(bins=5000, figsize=(8,4))
plt.xlim(-20, 1100)

In [ ]:
#avaliando quantas entregas foram atrasadas
atrasos = orders_diff_estimate_delivered < 0
atrasos.value_counts(normalize=True)

## avaliando os preços de frete
- Hipótese: produtos com frete mais barato tendem a ter mais vendas futuramente?

In [ ]:
# pegando a coluna de valor do frete
items = dfs['olist_order_items_dataset.csv']
df_merge = df_merge.merge(items[['order_id', 'freight_value']], on='order_id', how='left')

In [ ]:
df_merge['freight_value'].describe()

In [ ]:
# avaliando o bloxplot dos preços de frete por região
sns.boxplot(x='regiao', y='freight_value', data=df_merge)

## entendendo a relação de itens por pedidos

In [ ]:
print(f'qtd linhas df_orders: {len(orders)}, order_id distintos: {orders['order_id'].nunique()}')
print(f'qtd linhas df_items: {len(items)}, order_id distintos: {items['order_id'].nunique()}, order_item_id distintos: {items['order_item_id'].nunique()}')

In [ ]:
verifica_items = orders[['order_id']].merge(items[['order_id', 'product_id', 'price']])

In [ ]:
# agregando itens distintos por order_id
verifica_items = verifica_items.groupby(['order_id']).agg({'product_id': 'nunique'})
verifica_items.describe()


In [ ]:
verifica_items['product_id'].hist(bins=100)

In [ ]:
# identificando o percentual de pedidos com mais de um item
(verifica_items['product_id'] > 1).sum() / len(verifica_items)*100


A imensa maioria dos pedidos possuem apenas 1 item, apenas 3,27% deles têm mais de 1 item por pedido

## o mesmo item é vendido por diferentes vendedores?


In [ ]:
item_sellers = items.groupby(['product_id']).agg({'seller_id': 'nunique'})
item_sellers.describe()

In [ ]:
item_sellers['seller_id'].hist(bins=100)

In [ ]:
produtos_mais_um_vendedor = (item_sellers['seller_id'] > 1).sum()/len(item_sellers)*100
print(f'percentual de produtos com mais de um vendedor: {produtos_mais_um_vendedor:.2f}%')

A imensa maioria dos produtos não se repetem em diferentes vendedores, apenas 3,72% deles aparecem em diferentes vendedores

## quantidade de vendas por product_id
- Produtos com apenas 1 venda limitam a aplicação de variaveis explicatívas, por não ter oscilação ao longo do tempo para se avaliar.

In [ ]:
prod_counts = items.groupby("product_id")["order_id"].count().reset_index(name="n_vendas")
prod_counts['n_vendas'].describe()

In [ ]:
prod_counts.hist(bins=100)

In [ ]:
# Ajustar para limitar no máximo em 10 (10 ou mais)
prod_counts["n_vendas_limitado"] = prod_counts["n_vendas"].clip(upper=10)

freq = prod_counts["n_vendas_limitado"].value_counts(normalize=True).sort_index()*100
freq_df = freq.reset_index()
freq_df.columns = ["n_vendas", "frequencia"]
freq_df = freq_df.sort_values("n_vendas", ascending=False).reset_index(drop=True)
freq_df["frequencia_acumulada"] = freq_df["frequencia"].cumsum()
freq_df

É possível observar que 54,98% dos produtos possuem 1 única venda, enquando apenas 27,36% possuem 3 vendas ou mais.

## quantidade de vendas por quantidade de produtos vendidos
- Se a base é composta com produtos que possuem diversas vendas ao longo do tempo, é possível explorar os fatores que causam essa oscilação. Caso os produtos tenham a característica de 1 única venda, as exploração de variáveis explicativas fica limitada.

In [ ]:
# trás a visão
# x % das vendas acontecem por produtos vendidos 1 única vez
# x % das vendas acontecem por produtos vendidos 2 vezes ou mais
# ...

# 1) nº de vendas por produto
counts = items.groupby("product_id")["order_id"].count()

# 2) Buckets exatos: 1..9 e 10+ (rotulado como 10)
bucket = counts.clip(upper=10)

# 3) Vendas (soma de n_vendas) por bucket
sales_per_bucket = (
    pd.DataFrame({"n_vendas": counts, "bucket": bucket})
      .groupby("bucket", sort=True)["n_vendas"].sum()
    # garante todos os buckets de 1 a 10
    .reindex(range(1, 11), fill_value=0)
)

# 4) Percentual por bucket (exato) e acumulado de 10→1
total_sales = sales_per_bucket.sum()
pct = (sales_per_bucket / total_sales * 100).round(2)

idx_desc = list(range(10, 0, -1))  # 10,9,...,1
freq_df = pd.DataFrame({
    "n_vendas_min": idx_desc,                # 10 representa 10 ou mais
    "frequencia": [pct[i] for i in idx_desc] # % das vendas do bucket EXATO
})
freq_df["frequencia_acumulada"] = freq_df["frequencia"].cumsum().round(2)

freq_df


Do volume total de vendas, 73,59% acontecem por produtos que foram vendidos 3 vezes ou mais

## relação da avaliação dos usuário e número de vendas dos produtos
- Hipótese: produtos melhores avaliados possuem mais recorrência de vendas?

In [ ]:
# veficando o numero de linhas dos datasets que serão utilizados
print(len(items))
reviews = dfs['olist_order_reviews_dataset.csv']
print(len(reviews))

In [ ]:
# verficando o numero de linhas do dataset mergeado
items_reviews = items.merge(reviews, on='order_id', how='left')
print(len(items_reviews))
items_reviews.drop_duplicates()
print(len(items_reviews))

In [ ]:
# entender por que aumenta o numero de linhas
verifica_reviews = items_reviews.groupby(['order_id', 'product_id'])['review_id'].nunique()
verifica_reviews.reset_index().groupby('review_id').nunique()

# percebe-se que alguns casos possuem mais de uma avaliação, o que faz aumentar o

In [ ]:
# verificando os casos com mais de uma avaliação
id_vefificar = verifica_reviews[verifica_reviews == 3].reset_index()['order_id'].iloc[0]
items_reviews[items_reviews['order_id'] == id_vefificar]

In [ ]:
# avaliar a média das avaliações para cada faixa de n_vendas_min

# Média do review_score por faixa de n_vendas_min
# juntar com items_reviews (que tem 'product_id' e 'review_score')
# e aplicar a mesma bucketização
items_reviews = items_reviews.copy()
items_reviews["bucket"] = items_reviews["product_id"].map(bucket)

# calcular média de review_score por bucket
mean_reviews = (
    items_reviews.groupby("bucket")["review_score"]
    .mean()
    .reindex(range(1, 11))
    .round(2)
)

# adicionar a coluna na tabela final
freq_df["media_review_score"] = [mean_reviews.get(i, np.nan) for i in idx_desc]

freq_df

não há relação aparentes entre o numero de vendas e suas avaliações

## os produtos possuem variação de preço considerável ao longo do tempo?
- Hipótese: a variação de preços impacta nas vendas futuras?

In [ ]:
# filtrar produtos que tiveram 3 vendas ou mais
# ordenar pela data de venda
# calcular a variação percentual em relação à data mais recente

produtos_3_vendas =  prod_counts[prod_counts['n_vendas'] >= 3]['product_id']
orders_items = orders.merge(items, on='order_id', how='left')
variacao_precos = orders_items[orders_items['product_id'].isin(produtos_3_vendas)].sort_values('order_purchase_timestamp')
variacao_precos['price_pct_change'] = variacao_precos.groupby('product_id')['price'].pct_change() * 100

In [ ]:
# visualizando um exemplo em que ocorre variação
variacao_precos[variacao_precos['product_id'] == 'fffdb2d0ec8d6a61f0a0a0db3f25b441']

In [ ]:
# distribuição da variação
variacao_precos = variacao_precos.dropna(subset=['price_pct_change'])
variacao_precos['price_pct_change'].describe()

In [ ]:
variacao_precos['price_pct_change'].hist(bins=100)

In [ ]:
# percentual de casos com variacao diferente de 0
(variacao_precos['price_pct_change'] != 0).sum() / len(variacao_precos) * 100

Dentre os produtos que têm 3 vendas ou mais, 12% deles possuem variação de preço, então é possível explorar essa variável

# Visualização dos dados gerais

## Vendas e itens por categoria

In [ ]:
# order_id, item_id e categoria
df_items_products = items.merge(dfs['olist_products_dataset.csv'], on='product_id', how='left')

In [ ]:
df_items_products['product_category_name'].nunique()

In [ ]:
# gerando tabela com vendas e produtos por categoria
tabela_vendas_itens_categoria = df_items_products.groupby('product_category_name')\
.agg({'order_id': 'count', 'product_id': 'nunique'})\
.sort_values('order_id', ascending=False)\
.reset_index()

# lista com as 20 categorias mais vendidas
top20_categorias = tabela_vendas_itens_categoria['product_category_name'].loc[:19].to_list()

# mantendo o nome dos 20 primeiros produtos e atribuindo 'outros' para os demais
tabela_vendas_itens_categoria['product_category_name'] = tabela_vendas_itens_categoria['product_category_name'].apply(lambda x: x if x in top20_categorias else 'outros')

# reagrupando agora com a nova coluna
tabela_vendas_itens_categoria = tabela_vendas_itens_categoria.groupby('product_category_name')\
.agg({'order_id': 'sum', 'product_id': 'sum'})\
.sort_values('order_id', ascending=False)\
.reset_index()

# gerando coluna com média de vendas por produto
tabela_vendas_itens_categoria['vendas_por_item'] = (tabela_vendas_itens_categoria['order_id'] / tabela_vendas_itens_categoria['product_id']).round(2)

# renomeia as colunas
tabela_vendas_itens_categoria.columns = ['categoria', 'vendas', 'itens', 'vendas_por_item']

# coluna com o  percentual geral de vendas
tabela_vendas_itens_categoria['percentual_vendas'] = ((tabela_vendas_itens_categoria['vendas'] / tabela_vendas_itens_categoria['vendas'].sum()) * 100).round(2).astype(str) + "%"
tabela_vendas_itens_categoria




## Volumetria ao longo do tempo com agregação semanal

In [ ]:
# gerando tabela com vendas por semanas
pd.to_datetime(orders_items.order_purchase_timestamp).dt.date.hist(bins=593//7,figsize=(15,3))